# Data transformation

Machine learning (ML) algorithms may be sensitive to data representation, scale, and/or distribution. In this notebook, we'll discuss the main techniques provided by scikit-learn for this kind of preprocessing. To see how they work in practice, let's use a dataset for house price prediction provided by Kaggle. To download it, follow the [first stage of this tutorial](https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0), which shows how to download access credentials for Kaggle (`kaggle.json`). Once you have downloaded the credentials, use the side menu to upload the file to Colab, and run the cells below:

In [0]:
import pandas as pd
import seaborn as sns
sns.set()

In [0]:
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d prevek18/ames-housing-dataset

In [0]:
ames_housing = pd.read_csv("ames-housing-dataset.zip")
ames_housing.head()

> Should any of the cells above fail, contact the maintainers of scikit-zero ;)

Peeking the first rows we can see that this is a very large dataset w.r.t. number of features. Let's check how many samples it has:

In [0]:
ames_housing.shape

> Kinda like iris, right? 🤣

Let's start isolating the feature we would like to predict from the input features:

In [0]:
X = ames_housing.drop(["SalePrice"], axis=1)
y = ames_housing["SalePrice"]

> Note that the feature we want to predict is real-valued, so we have a **regression** problem.

To make our analysis a bit more practical for the purposes of this notebook, we'll **filter out** features according to some pre-defined criteria:
* features that are clearly unrelated to the target feature;
* features that present more than 90% of missing values;
* features with low variance:
    * Categorical features: the most ocurring value occurs in more than 90% of the samples.
    * Numerical features: the 90% quantile matches the median.
    
> Note that this filtering is specific to this notebook and could likely represent a premature decision in the machine learning modeling process. We do it here make the dataset a bit simpler. For this reason also, we pre-identified these features rather than computing them in the notebook.

In [0]:
unrelated = ["Order", "PID", "MS SubClass"]
many_missing = ["Pool QC", "Misc Feature", "Alley"]
low_variance_nominal = ['Paved Drive', 'Electrical', 'Functional', 'Central Air', 'Land Slope', 'Heating', 'Roof Matl', 'Condition 2']
low_variance_ordinal = ['Garage Cond', 'Street', 'Utilities']
low_variance_numerical = ["Low Qual Fin SF", "Bsmt Half Bath", "Full Bath", "Kitchen AbvGr",
                          "Fireplaces", "3Ssn Porch", "Screen Porch", "Pool Area", "Misc Val"]

In [0]:
X = X.drop(unrelated + many_missing +
           low_variance_nominal + low_variance_ordinal,
           axis=1)

> Even with this filtering, this dataset is still quite complex. In real life, we would never go for modelling before having conducted an exploratory data analysis. Here, we are making things a bit simplistic since we're skipping that, but bear with me :)

In [0]:
X.shape

## Encoding ordinal features

To properly prepare the data for modelling, the first thing we need to do is isolate feature types. This is very important, since different types of feature demand different preparation techniques. From a quick look at the data, I have isolated the following three feature subsets:

> If you don't understand the difference between feature types, check [pandas-zero](https://github.com/leobezerra/pandas-zero). 

> If you do, notice that a few ordinal features are being listed as nominal. The reason is that we do not have a rich data dictionary available, and for those features it was not straightforward to understand the ranking implied by the values.

In [0]:
nominal = ["MS Zoning", "Lot Shape", "Land Contour",  "Lot Config", 
           "Neighborhood", "Condition 1", "Bldg Type", "House Style", 
           "Roof Style", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", 
           "Foundation", "BsmtFin Type 1", "BsmtFin Type 2", "Garage Type", 
           "Fence", "Sale Type", "Sale Condition", "Bsmt Exposure", "Garage Finish"]

ordinal = ["Exter Qual", "Exter Cond", "Bsmt Qual", "Bsmt Cond",  "Heating QC",
           "Kitchen Qual", "Fireplace Qu", "Garage Qual"]

numerical = ["Lot Frontage", "Lot Area", "Year Built", "Year Remod/Add", "Mas Vnr Area",
             "BsmtFin SF 1", "BsmtFin SF 2", "Bsmt Unf SF", "Total Bsmt SF", "1st Flr SF",
             "2nd Flr SF", "Low Qual Fin SF", "Gr Liv Area", "Bsmt Full Bath", "Bsmt Half Bath", 
             "Full Bath", "Half Bath", "Bedroom AbvGr", "Kitchen AbvGr", "TotRms AbvGrd",
             "Fireplaces", "Garage Yr Blt", "Garage Cars", "Garage Area", "Wood Deck SF",
             "Open Porch SF", "Enclosed Porch", "3Ssn Porch", "Screen Porch", "Pool Area",
             "Misc Val", "Mo Sold", "Yr Sold", "Overall Cond", "Overall Qual"]

Let's temporarily isolate the ordinal features to see how we can prepare them:

In [0]:
X_ordinal = X[ordinal]
X_ordinal.head()

Checking these features a bit we see that they follow a ranking pattern:
0. `TA`: to be assessed
1. `Po`: poor
2. `Gd`: good
3. `Ex`: excellent
4. `Fa`: fantastic or fabulous (I guess)

The encoding we apply to ordinal features is converting each ranking category into an integer, so that higher ranks are represented by larger values. 

> `TA` is equivalent to a missing value, and we'll rank it 0. Again, this could be premature, but it's a reasonable choice here.

Encoding ordinal features in scikit-learn can be done with the `OrdinalEncoder` preprocessor from the `preprocessing` module:

In [0]:
from sklearn.preprocessing import OrdinalEncoder

`OrdinalEncoder` takes as input a list of lists: 
- one list per feature
- each list should indicate the ranking of the categories

For our example, the same list can be used to indicate the ranking of the categories:

In [0]:
conditions = ["TA", "Po", "Gd", "Ex", "Fa"]

One limitation of `OrdinalEncoder` is that we cannot indicate that the same list will be used for multiple features. To replicate the list as many times as the number of features we have, we're gonna use the list comprehension notation we've discussed back in pandas-zero:

> the `for i in range(n_features)` means we will have `n_features` repetitions of `conditions`

In [0]:
n_features = X_ordinal.shape[1]
condition_lists = [conditions for i in range(n_features)]c

We can now create our encoder for ordinal features:

In [0]:
condition_encoder = OrdinalEncoder(categories=condition_lists)

Another limitation of `OrdinalEncoder` is that is cannot be applied to data with missing values. For this reason, we're gonna use a pipeline where we first impute `'TA'` to the missing values, and then apply the encoding:

In [0]:
from sklearn.impute import SimpleImputer
condition_imputer = SimpleImputer(strategy="constant", fill_value="TA")

In [0]:
from sklearn.pipeline import make_pipeline
ordinal_pipe = make_pipeline(condition_imputer, condition_encoder)

When we use a pipeline solely for data preparation, we use `fit()` and `transform()`, rather than `fit()` and `predict()`. In this case, we'll make it simpler and use `fit_transform()` directly:

> Note that we wrap the output of the pipeline in a `DataFrame` object and preserve the column values.

In [0]:
X_ordinal = pd.DataFrame(ordinal_pipe.fit_transform(X_ordinal), columns=X_ordinal.columns)
X_ordinal.head()

Now our ordinal features have been encoded properly, and we can move on to nominal features.

## Encoding nominal features

Once again, let's start isolating our nominal features:

In [0]:
X_nominal = X[nominal]
X_nominal.head()

In [0]:
X_nominal.shape

The most common approach to nominal feature encoding is called **one-hot encoding**. Let's take feature `Lot Shape` as example:

In [0]:
X_nominal["Lot Shape"].value_counts(dropna=False)

With one-hot encoding, feature `Lot Shape` gets replaced by binary features, one for each of the possible values presented by `Lot Shape`. Let's start importing the scikit-learn resources we're gonna need:

In [0]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

The `make_column_transformer` from the `compose` module allows us to specify encodings for specific features.

- Let's understand the code below:
```python 
nominal_encoder = make_column_transformer(
                                            (OneHotEncoder(sparse=False), ["Lot Shape"]),
                                            remainder="drop")
```
> The `sparse=False` attribute indicates that the output should be a regular numpy array, rather than a more efficient scipy sparse matrix. Even if the former is less efficient, it is more readable and helps keep notebook understandable.

- Creates the column transformer, specifying that a `OneHotEncoder()` should be applied to `["Lot Shape"]`, and that the remainder of the features should be dropped. 
```python 
encoded_data = nominal_encoder.fit_transform(X_nominal)
```
- Applies the `fit_transform()` method to `X_nominal`. 
```python
encoded_data = pd.DataFrame(encoded_data,
                              columns=nominal_encoder.get_feature_names())
```
- Wrap the numpy array as a DataFrame, retrieving the feature names created by the encoder.

Let's see the output of that:

In [0]:
nominal_encoder = make_column_transformer(
                                          (OneHotEncoder(sparse=False), ["Lot Shape"]),
                                          remainder="drop")
encoded_data = nominal_encoder.fit_transform(X_nominal)
encoded_data = pd.DataFrame(encoded_data,
                            columns=nominal_encoder.get_feature_names())
encoded_data.head()

Note that the features are now labeled as a function of the feature values. For each sample, only one of the enconded feature values equals 1.0, since that was the original value before the encoding.

> You can imagine that this type of encoding explodes the number of features in a dataframe, right? Let's see by how much!

In [0]:
nominal_pipe = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(sparse=False))
pd.DataFrame(nominal_pipe.fit_transform(X_nominal),
             columns=nominal_pipe.steps[1][1].get_feature_names(nominal))

> Note that we could have used better strategies than a `SimpleImputer(strategy='most_frequent')`, but that would make the notebook even longer 🙃

## Preparing numerical features

When dealing with numerical features, our major concerns regard the data scale and distribution. Scikit-learn offers several resources to address these issues, as we'll see below.

### Feature scaling

A very important aspect in data preprocessing concerns feature scaling. Take an algorithm that considers Euclidean distances, such as kNN. If a feature presents much larger values than another, the relative importance of these features will be unbalanced. Sometimes, this comes in your favor, since the features with larger values are indeed the most important. Other times, your model will be undervaluing the most relevant features.

Several scaling techniques can be used to make different features present the same scale, but they only make sense when we're talking about numerical features. For now, let's isolate the numerical features from the remaining. 

In [0]:
X_num = X[numerical]
X_num.head()

In [0]:
X_num.shape

Note that now we're working with 35 features. Let's move on to the techniques so we can understand our options.

#### Fitting to a range

The first thing we notice about the numerical features of this dataset is that feature ranges are very different. Let's check, for instance, the range differences between features `Overall Qual`, `Year Built`, and `Gr Liv Area`:

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

We can scale all features to the `[0,1]` range using the `MinMaxScaler` preprocessor:

In [0]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X_num = pd.DataFrame(minmax.fit_transform(X_num), columns=X_num.columns)
X_num.head()

Note that now the min and max values for each feature now match:

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

> Note how the range scaling made the means of `Year Built` and `Gr Liv Area` different, when they were originally very close!

#### Centering the data

Another transformation commonly applied to numerical data is centering the data so it presents zero mean. We can do this transformation with using the `StandardScaler` preprocessor, following the same pattern adopted above:

In [0]:
from sklearn.preprocessing import StandardScaler
zero_mean = StandardScaler(with_std=False)
X_num = pd.DataFrame(zero_mean.fit_transform(X_num), columns=X_num.columns)
X_num.head()

> The parameter `with_std` defines whether `StandardScaler` also scales the variance. Here we disabled it not to spoil the next topic 😅

After the transformation, all features present zero mean:

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

> Computers have a big trouble matching exact numbers when dealing with real values. The values shown in the mean column above are small enough to be considered zero by most algorithms.

#### Scaling the variance

A final transformation that is often applied to numerical data is scaling the variance of the data to one. 

> It is so common that the `StandardScaler` preprocessor does it by default! Since we're doing it step-wisely, we will tell the preprocessor it doesn't need to center the data again (`with_mean=False`).

In [0]:
unit_variance = StandardScaler(with_mean=False)
X_num = pd.DataFrame(unit_variance.fit_transform(X_num), columns=X_num.columns)
X_num.head()

After the transformation, all features present unit variance:

> And, by definition, unit standard deviation!

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].var()

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

#### Using pipelines

As discussed in the first steps with scikit-learn, pipelines are the best way to combine all the steps in a machine learning experiment. Here, we're gonna use them to encapsulate all the transformations applied to numerical features:

In [0]:
from sklearn.impute import KNNImputer
num_pipe = make_pipeline(KNNImputer(), MinMaxScaler(), StandardScaler())

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

In [0]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

### Transforming distributions

The scaling conducted above does not affect the distribution shape of the data. From many aspects, having the data distributed according to a normal distribution is desirable. 

> In pandas-zero, we discussed how to transform a logarithmic distribution into a normal distribution.

> We also discussed how to check if a distribution differs from a normal distribution. Tip: the second value in each parenthesis should be greater than or equal to 0.05.

In [0]:
from scipy.stats import normaltest
X_num.apply(normaltest)

Scikit-learn offers two preprocessing components to transform distributions into normal distributions, which we discuss next. 

#### Quantization

Quantization is the act of replacing a feature value for the quantile it belongs to. This is a non-parametric transformation that can be mapped to different distributions. scikit-learn provides the `QuantileTransformer` preprocessor, where we configure the desired output distribution using the `output_distribution` argument:

In [0]:
from sklearn.preprocessing import QuantileTransformer
num_pipe = make_pipeline(KNNImputer(),
                         MinMaxScaler(),
                         StandardScaler(),
                         QuantileTransformer(output_distribution='normal'))

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

In [0]:
X_num.apply(normaltest)

From the results above, we see that only the `Yr Sold` feature is now matched with a normal distribution.

> Note, however, that the p-values (second value in each parenthesis) great increases for most features, indicating that the transformed distributions are closer to a normal distribution than the original ones.

#### Power transformations

These parametric transformations are similar in spirit to the logarithmic transformation we discussed in pandas-zero. scikit-learn provides two options within the `PowerTransformer` preprocessor:

* the Yeo-Johnson transform, which works for positive and negative values;
* the Box-Cox transform, which only works for strictly positive values.

Since we have centered the data, we have to use the Yeo-Johnson transform (the default method for `PowerTransform`):

In [0]:
from sklearn.preprocessing import PowerTransformer
num_pipe = make_pipeline(KNNImputer(), 
                         MinMaxScaler(),
                         StandardScaler(),
                         PowerTransformer())

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

In [0]:
X_num.apply(normaltest)

This time, `Yr Sold` does not match a normal distribution. Yet, a larger number of features do, namely `Overall Qual`, `1st Flr SF`, `Gr Liv Area`, and `TotRms AbvGrd`. 

## Wrapping it all

Although we should always deal with features according to their type, we can prepare our data without having to split our dataset as we did it above:

In [0]:
full_pipe = make_column_transformer(
                                    (ordinal_pipe, ordinal),
                                    (nominal_pipe, nominal),
                                    (num_pipe, numerical)
                                    )

In order to access feature names produced by transforming nominal features, we'll fit that pipeline to the data, even if we're not gonna use it to transform the data:

In [0]:
nominal_pipe.fit(X[nominal])
nominal_encoder = nominal_pipe.steps[1][1] # this access the OneHotEncoder within the pipeline
nominal_transformed = nominal_encoder.get_feature_names(nominal).tolist()

The feature names for the whole dataset will be the concatenation of the following features:

In [0]:
column_names = ordinal + nominal_transformed + numerical

We can now transform the data, preserving the dataframe feature names and therefore its readibility:

In [0]:
X_transformed = pd.DataFrame(full_pipe.fit_transform(X),
                             columns=column_names)
X_transformed.head()